In [1]:
scene_ = ["ESC:", "SEC:", "ESCENA:", "ESCENA", "SECUENCIA:", "SECUENCIA"]
in_out_ = ["INT./EXT", "EXT./INT","INT", "EXT"]
moment_ = ["TARDE-NOCHE","DÍA","UN RATO DESPUÉS" ,"DIA", "AMANECER", "MAÑANA", "TARDE", "ATARDECER", "ANOCHECER", "NOCHE", "MADRUGADA", "OCASO", "MÁS TARDE", "ALBA", "TARDE/OCASO", "AL DÍA SIGUIENTE", "MOMENTOS DESPUÉS", "CONTINÚA"]

In [2]:
scene_options = sorted(scene_, key=len, reverse=True)
in_out_options = sorted(in_out_, key=len, reverse=True)
moment_options = sorted(moment_, key=len, reverse=True)

In [3]:
Scene2 = '|'.join(scene_options)
In_out2 = '|'.join(in_out_options)
Moment2 = '|'.join(moment_options)

In [27]:
from sly import Lexer, Parser



class MyLexer(Lexer):
    tokens={SEPARADOR, SCENE_OPTIONS, IN_OUT_OPTIONS, MOMENT_OPTIONS, NUMBER,TEXT}

    ignore=' \t\n'

    SCENE_OPTIONS=rf'{Scene2}'
    IN_OUT_OPTIONS = rf'{In_out2}'
    MOMENT_OPTIONS = rf'{Moment2}'
    NUMBER = r'\d+'
    SEPARADOR = r'\-|\-\-|/|\.'
    TEXT = r'[^\s\n\t]+'

class MyParser(Parser):
    tokens = MyLexer.tokens

    # Regla principal de la gramática que reconoce la estructura general
    @_('opcional_escena opcional_numero in_out_opcion texto_con_separadores moment_options')
    def entrada(self, p):
        return ( p.opcional_escena, p.opcional_numero, p.in_out_opcion, p.texto_con_separadores, p.moment_options)

    # Reglas para manejar la opción de tener o no SCENE_OPTIONS
    @_('SCENE_OPTIONS')
    def opcional_escena(self, p):
        return p.SCENE_OPTIONS

    @_('empty')
    def opcional_escena(self, p):
        return None

    # Reglas para manejar la opción de tener o no NUMBER
    @_('NUMBER SEPARADOR')
    def opcional_numero(self, p):
        return p.NUMBER

    @_('empty')
    def opcional_numero(self, p):
        return None

    # Regla para manejar IN_OUT_OPTIONS seguido de un separador
    @_('IN_OUT_OPTIONS SEPARADOR')
    def in_out_opcion(self, p):
        return p.IN_OUT_OPTIONS

    @_('TEXT secuencia_de_texto')
    def texto_con_separadores(self, p):
        return f"{p.TEXT} {p.secuencia_de_texto}"
    
    @_('SEPARADOR TEXT secuencia_de_texto ')
    def secuencia_de_texto(self, p):
        return f"{p.SEPARADOR} {p.TEXT} {p.secuencia_de_texto}"
    
    @_('TEXT secuencia_de_texto')
    def secuencia_de_texto(self, p):
        return f"{p.TEXT} {p.secuencia_de_texto}"

    @_('TEXT SEPARADOR')
    def secuencia_de_texto(self, p):
        return p.TEXT
    
    @_('SEPARADOR')
    def secuencia_de_texto(self, p):
        return ''
    
    @_('MOMENT_OPTIONS')
    def moment_options(self, p):
        return p.MOMENT_OPTIONS

    # Regla para manejar casos vacíos (ausencia de SCENE_OPTIONS o NUMBER)
    @_('')
    def empty(self, p):
        return None

    # Manejo de errores sintácticos
    def error(self, p):
        if p:
            print(f"Error de sintaxis en el token {p.type}, valor {p.value}")
        else:
            print("Error de sintaxis en el final de la entrada")



In [28]:
lexer = MyLexer()
parser = MyParser()


examples = [
        "ESC: 1. INT. APARTAMENTO/ CUARTO HUGO / DÍA"
    ]

for data in examples:
        token=lexer.tokenize(data)
        # for tok in token:
        #         print(tok)
        result = parser.parse(token)
        print(result)

('ESC:', '1', 'INT', 'APARTAMENTO/ CUARTO HUGO', 'DÍA')


In [7]:
class Scene(object):
    def __init__(self, number, in_out, place, moment, time, page,characters,continuity,text):
        self.number = number
        self.in_out = in_out
        self.place = place
        self.moment = moment
        self.time = time
        self.page = page
        self.characters=characters
        self.continuity=continuity
        self.text=text

In [8]:
class Scene_separator(object):
    def __init__(self, scene_number_separator, number_in_out_separator, in_out_place_separator, moment_separator, scene_options, in_out_options, moment_options):
        self.Scene_number_separator = scene_number_separator
        self.Number_in_out_separator = number_in_out_separator
        self.In_out_place_separator = in_out_place_separator
        self.Moment_separator = moment_separator
        self.Scene_options = scene_options
        self.In_out_options = in_out_options
        self.Moment_options = moment_options
  
    def __call__(self, script_text_per_page):
        scenes_headings = []
        past_line=''
        index_scene=1
        first_sce=False
        text=''
        old_result=(None,None,None,None,None)
        
        for page in script_text_per_page.keys():
            page_content = script_text_per_page[page]
            page_content = page_content.split('\n')

            for line in page_content:

                result = parser.parse(lexer.tokenize(line))

                if result is None and not first_sce:
                    result3=parser.parse(lexer.tokenize(past_line+" "+line))
                    if result3 is None:
                        past_line=line
                    else:
                        old_result=result3
                        past_line=''
                        first_sce=True
                elif result is None:
                    result2 = parser.parse(lexer.tokenize(past_line+" "+line))
                    if result2 is not None:
                        scenes_headings.append(Scene(index_scene,old_result[2],old_result[3],old_result[4],self.calculate_time(len(text)),page,None,None,text))
                        text=''
                        index_scene+=1
                        old_result=result2
                        past_line=''
                    else:
                        text=text + "\n"+past_line
                        past_line=line
                else:
                    text = text + past_line
                    past_line=''
                    if result[1] is None:
                        scenes_headings.append(Scene(index_scene,old_result[2],old_result[3],old_result[4],self.calculate_time(len(text)),page,None,None,text))
                    else:
                        scenes_headings.append(Scene(old_result[1],old_result[2],old_result[3],old_result[4],self.calculate_time(len(text)),page,None,None,text))
                    text=''
                    index_scene+=1
                    old_result=result
            
        if old_result != (None,None,None,None,None):
            text=text + "\n"+past_line
            if result[1] is None:
                scenes_headings.append(Scene(index_scene,old_result[2],old_result[3],old_result[4],self.calculate_time(len(text)),page,None,None,text))
            else:
                scenes_headings.append(Scene(old_result[1],old_result[2],old_result[3],old_result[4],self.calculate_time(len(text)),page,None,None,text))
                
        return scenes_headings


    @staticmethod
    def calculate_time(char_count):
        # quotient, remainder = divmod(a, b)
        minutes, seconds = divmod(char_count, 1020)
        return (minutes, seconds)

In [9]:
data='''fsbhds dashdbsahjf dshdbsahjfsad dsdbsajdsad
dbsahdhjsabdhjsbd shdjbshajdhsajbd shjdbhsajdbhj shjdbhsajdb
kjdhjsabdjhsad sjdsajdhjsbd jhasdbhsjabdhjd

ESC: 1. INT. APARTAMENTO HUGO Y SIMONE. CUARTO HUGO - DÍA
Habitación de adolescente. Una cama personal, una computadora
de escritorio. Pegados en las paredes varios afiches
ambientados en mundos de fantasía épica, con mujeres
semidesnudas, magos y dragones. También afiches de bandas de
rock épico. En unos estantes, figuritas de acción de dragones
y caballeros con espadas. Un librero pequeño con libros como
«El Silmarillion» y «Canción de hielo y fuego».
En una esquina dos o tres maletines abiertos de los que
sobresale ropa, zapatos, etc. Todo muy regado.
Sobre la cama está acostado HUGO (17), un muchacho flacucho y
no muy agraciado. A su alrededor, varios papeles de estadística
de personajes de un juego de rol. Mantiene por WhatsApp una
partida de juego de rol con sus amigos.
HUGO
(narra con atmósfera de
misterio)
Se adentran en el pantano. El olor
a podredumbre y azufre es
insoportable. A su paso tropiezan
con espadas melladas, escudos
quebrados, y esqueletos de antiguos
guerreros... Caminando encuentran
la cabaña de las tres brujas que
poseen la mitad del pergamino. La
puerta de la cabaña está clausurada
por cadenas... ¿Qué hacen?
MUCHACHO 1 (O.S.)
Uso el poder psíquico de mi
hechicera de hielo para contener a
las brujas, y que el paladín abra
la puerta.
HUGO
Hagamos la tirada.
Hugo usa una app en el móvil para hacer tiradas de dados
digitales. Sale un 3.
CORO AL OTRO LADO DEL TELÉFONO (O.S.)
(emocionados)
¡Nooo!
HUGO
(narra)
El poder de tu hechicera no logra
contener a las brujas, y caes
aturdido por dos turnos... Las
brujas rompen la puerta y la líder
grita enfurecida:
Repentina, abre la puerta AURORA (45), madre de Hugo y
Simone.
AURORA
(alterada)
¡Huguito, deja la bobería y acaba
de recoger!

ESC: 2. INT. APARTAMENTO HUGO Y SIMONE. SALA - DÍA 
Aurora cierra la puerta del cuarto de Hugo y camina a la
sala. Está apresurada.
Su hija SIMONE (15) (se pronuncia Simón) le sale al paso. La
muchacha lleva el pelo corto y despeinado. No usa aretes.
Viste con un jean y una blusa blanca de hilo.
SIMONE
(angustiada)
Mami, no nos hagas esto.
Aurora la ignora.
Se acerca MARCOS (47), el padre de los muchachos, y esposo de
Aurora. Un tipo serio, pero noble.
MARCOS
(a Aurora)
Voy sacando el carro
AURORA
Primero saca a Hugo del cuarto.
Marcos respira como buscando paciencia. Va hacia el cuarto de
Hugo.
Aurora va hasta la cocina. Simone le cae detrás.

ESC: 3. INT. APARTAMENTO HUGO Y SIMONE. COCINA 
- DÍA
Simone detiene a Aurora en la entrada de la cocina.
SIMONE
Mami, por favor.
Marcos sale del cuarto con el celular de Hugo en la mano.
Hugo va detrás de él, reclamando el teléfono para continuar
el juego.
Pasan junto a Aurora y Simone.
HUGO
Papá, que estamos atacando a las
brujas del pantano.
SIMONE
(a Aurora)
Te lo suplico.
AURORA
Simón, ¿Y qué quieres que haga?
¿Dime?
SIMONE
Que me lleves a París con ustedes.
Hugo retrocede y se mete en la conversación.
HUGO
(a Simone)
Los trasgos caníbales son menos
traicioneros que tú.
SIMONE
(a Aurora, ignorando a
Hugo)
Esto sería como un regalo por mis
quince.
Aurora revisa cosas en la cocina; que el gas esté cerrado,
que el cesto de basura esté limpio, etc.'''

In [10]:
def is_header(line):
    return parser.parse(lexer.tokenize(line))

In [11]:
scenes_headings = []
past_line=""
index_scene=1
text=""
old_result=None
        

page_content = data.split('\n')

for line in page_content:

    result = is_header(line.strip())
    if old_result is None:
        if result is None:
            past_line+=" "
            result2= is_header(past_line+line.strip())
            if result2 is None:
                past_line=line.strip()
            else:
                old_result=result2
                past_line=""
        else:
            old_result=result
            past_line=""
        continue
    elif result is None:
        past_line+=" "
        result3 = is_header(past_line+line.strip())
        if result3 is None:
            text+= "\n"+ past_line.strip()
            past_line=line.strip()
        else:
            if old_result[1] is None:
                scenes_headings.append(Scene(index_scene,old_result[2],old_result[3],old_result[4],None,None,None,None,text.strip()))
            else:
                scenes_headings.append(Scene(old_result[1],old_result[2],old_result[3],old_result[4],None,None,None,None,text.strip()))
            past_line=""
            text=""
            old_result=result3
            index_scene+=1
    else:
        text+= "\n"+ past_line.strip()
        if old_result[1] is None:
            scenes_headings.append(Scene(index_scene,old_result[2],old_result[3],old_result[4],None,None,None,None,text.strip()))
        else:
            scenes_headings.append(Scene(old_result[1],old_result[2],old_result[3],old_result[4],None,None,None,None,text.strip()))
        past_line=""
        text=""
        old_result=result
        index_scene+=1
            
if old_result != None:
    text+= "\n"+past_line.strip()
    if old_result[1] is None:
        scenes_headings.append(Scene(index_scene,old_result[2],old_result[3],old_result[4],None,None,None,None,text.strip()))
    else:
        scenes_headings.append(Scene(old_result[1],old_result[2],old_result[3],old_result[4],None,None,None,None,text.strip()))

NameError: name 'parser' is not defined

In [66]:
for sc in scenes_headings:
    print(f'LA ESCENA NUMEROOOOOO    {sc.number} {sc.in_out} {sc.place} {sc.moment} {sc.text}')

LA ESCENA NUMEROOOOOO    1 INT APARTAMENTO HUGO Y SIMONE. CUARTO HUGO DÍA Habitación de adolescente. Una cama personal, una computadora
de escritorio. Pegados en las paredes varios afiches
ambientados en mundos de fantasía épica, con mujeres
semidesnudas, magos y dragones. También afiches de bandas de
rock épico. En unos estantes, figuritas de acción de dragones
y caballeros con espadas. Un librero pequeño con libros como
«El Silmarillion» y «Canción de hielo y fuego».
En una esquina dos o tres maletines abiertos de los que
sobresale ropa, zapatos, etc. Todo muy regado.
Sobre la cama está acostado HUGO (17), un muchacho flacucho y
no muy agraciado. A su alrededor, varios papeles de estadística
de personajes de un juego de rol. Mantiene por WhatsApp una
partida de juego de rol con sus amigos.
HUGO
(narra con atmósfera de
misterio)
Se adentran en el pantano. El olor
a podredumbre y azufre es
insoportable. A su paso tropiezan
con espadas melladas, escudos
quebrados, y esqueletos de antigu